In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
# from tqdm.auto import tqdm
from tqdm import tqdm
import pandas as pd
import torch
from scipy import stats
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [71]:
student_id = 21522188

In [138]:
torch.randint(10, size=(10,))

tensor([8, 6, 5, 1, 4, 9, 7, 2, 0, 7])

# Objective Functions

In [165]:
def f1(pop):
    """Sphere Function"""
    return (pop**2).sum(-1)


def f2(pop):
    "Griewank Function"
    pop_size, d = pop.shape

    sum1 = (pop**2).sum(-1) / 4000

    sqrt_indices = (
        torch.sqrt(torch.arange(1, d + 1, device=pop.device))
        .unsqueeze(0)
        .expand((pop_size, d))
    )

    cos1 = torch.cos(pop / sqrt_indices).prod(-1)

    return sum1 - cos1 + 1


def f3(pop):
    """Rosenbrock Function"""

    pop0 = pop[:, :-1]
    pop1 = pop[:, 1:]

    return (100 * ((pop1 - pop0**2) ** 2) + (pop0 - 1) ** 2).sum(-1)


def f4(pop, m=10):
    """Michalewicz Function"""
    pop_size, d = pop.shape

    indices = (
        torch.arange(1, d + 1, device=pop.device).unsqueeze(0).expand((pop_size, d))
    )

    sin1 = torch.sin(indices * (pop**2) / np.pi) ** (2 * m)

    return -(torch.sin(pop) * sin1).sum(-1)


def f5(pop, a=20, b=0.2, c=2 * np.pi):
    """Ackley Function"""
    pop_size, d = pop.shape

    exp_mean_cos = torch.exp(torch.cos(pop * c).sum(-1) / d)

    exp_sqrt_mean_square = torch.exp(torch.sqrt((pop**2).sum(-1) / d) * (-b))

    return exp_sqrt_mean_square * (-a) - exp_mean_cos + a + np.e


objectives = {
    1: {"name": "Sphere Function", "domain": [-5.12, 5.12], "func": f1},
    2: {"name": "Griewank Function", "domain": [-600, 600], "func": f2},
    3: {"name": "Rosenbrock Function", "domain": [-5, 10], "func": f3},
    4: {"name": "Michalewicz Function", "domain": [0, np.pi], "func": f4},
    5: {"name": "Ackley Function", "domain": [-32.768, 32.768], "func": f5},
}

In [68]:
N = [16, 32, 64, 128, 256]

2.718281828459045

# differential evolution

In [196]:
def initialize_population(num_individuals, d, low=0, high=1, device="cpu"):
    pop = torch.distributions.uniform.Uniform(low, high).sample((num_individuals, d))
    return pop


def create_mutant_pop(pop, F=0.1):
    pop_size, d = pop.shape

    # sample 3 indices from {0,1,2,...,N-1} / {i}
    uniform_dis = torch.ones((pop_size, pop_size - 1), device=pop.device)

    r = uniform_dis.multinomial(num_samples=3, replacement=False)

    r = torch.where(
        r >= torch.arange(pop_size, device=pop.device).unsqueeze(1), r + 1, r
    ).T

    v = pop[r[0]] + F * (pop[r[1]] - pop[r[2]])

    return v


def create_trial_pop(pop, Cr, F=0.1):
    pop_size, d = pop.shape

    # to ensure that offspring is different to their parent
    Jrand = torch.randint(d, size=(pop_size,), device=pop.device)

    # convert to the actual indices after flatten
    Jrand += torch.arange(pop_size, device=pop.device) * d

    condition = (torch.rand((pop_size, d), device=pop.device) <= Cr).flatten()

    condition[Jrand] = True

    condition = condition.reshape((pop_size, d))

    V = create_mutant_pop(pop, F)

    U = torch.where(condition, V, pop)

    return U


def de_selection(pop, trial_pop, pop_fitness, trial_pop_fitness):
    selected = trial_pop_fitness < pop_fitness

    new_pop = torch.where(selected.unsqueeze(1).expand_as(pop), trial_pop, pop)
    new_fitness = torch.where(selected, trial_pop_fitness, pop_fitness)

    return new_pop, new_fitness


def differential_evolution(
    objective=objectives[1],
    N=16,
    d=2,
    Cr=0.1,
    F=0.1,
    max_evaluations=None,
    seed=49,
    device="cpu",
):
    torch.manual_seed(seed)

    if max_evaluations is None:
        max_evaluations = d * 10000

    pop = initialize_population(
        num_individuals=N,
        d=d,
        low=objective["domain"][0],
        high=objective["domain"][1],
        device=device,
    )
    pop_fitness = objective["func"](pop)
    num_evaluations = N

    # generation
    g = 0

    generation_log = []

    while num_evaluations < max_evaluations:
        best_idv = pop_fitness.argmax()
        generation_log.append([
            num_evaluations,
            pop_fitness[best_idv].cpu().item(),
            pop[best_idv].cpu().tolist(),
        ])

        # create trial population
        trial_pop = create_trial_pop(pop=pop, Cr=Cr, F=F)
        trial_pop_fitness = objective["func"](trial_pop)
        num_evaluations += N

        pop, pop_fitness = de_selection(
            pop, trial_pop, pop_fitness, trial_pop_fitness
        )

        g += 1


    best_idv = pop_fitness.argmax()
    generation_log.append([
        num_evaluations,
        pop_fitness[best_idv].cpu().item(),
        pop[best_idv].cpu().tolist(),
    ])
    
    return generation_log

In [198]:
%%time

re = differential_evolution(
    objective=objectives[i],
)


CPU times: user 297 ms, sys: 1.99 ms, total: 299 ms
Wall time: 301 ms


# CEM